# Zero to Hero : An Overwatch Hero Recommender

# Table of Contents:

1. **Data and Functions Import**  
2. **Cleaning and Reformating Data into Relevant Structure**  
3. **Role Category Implementation**  
4. **Implementing the Recommendation System**  
    4.1 Training the model  
5. **Making a Recommendation**  
    5.1 Interactive user_rating input  
    5.2 Making predictions with new user ratings

## Executive Summary
Zero to Hero is a recommendation system that offers users who play the popular eSports title: Overwatch a way to choose a hero to play with. In Overwatch, two teams of 6 players go head to head in a heated contest. With a growing pool of 31 heroes to choose from in 3 different roles, new and veteran users can benefit from this hero recommender. 

Often players will be stuck single-mindedly on a hero unwilling to choose a new hero to play. This can put enormous pressure on the team. These users will be able to get a hero recommended to them using Zero to hero. Older players who have played a long time and want to diversify their hero pool can also benefit. 

The system falls in line with the game developers encouragement to players of choosing heroes that they enjoy playing or at the very least providing some variety in their hero choices.

## 1. Data and Functions Import

In [1]:
# Import all local functions from the source py file:
from recommender_utils import *

ModuleNotFoundError: No module named 'surprise'

In [ ]:
# Read in the top5 heroes data for 60k users:
top5 = pd.read_csv('60k_Top5_heroes.csv')
top5

## 2. Cleaning and formatting Data

In [2]:
# Apply preprocessing and curation to clean missing values, format data as Dummies
# and apply ratings to the top5 heroes:
heroes_preprocessed = preprocessing(top5)
heroes_curated = apply_curation(heroes_preprocessed)

NameError: name 'preprocessing' is not defined

In [4]:
# Melt the dummy data into the corect format for the recommendation engine:
heroes = concat_melt(heroes_curated)
# Rename the columns of the dataframe:
heroes.rename(columns={"variable":"Hero", "value":"Ratings"}, inplace = True)
heroes

,Battle_Tag,Hero,Ratings
0,Morellio-1416,Ana,0.0
1,Zinux-1251,Ana,4.8
2,BluePillow-21338,Ana,0.0
3,Miyuna-21516,Ana,0.0
4,Mood-21538,Ana,4.4
...,...,...,...
1870783,Dot-11883,Zenyatta,0.0
1870784,hanayarawa55-1556,Zenyatta,4.8
1870785,blowedglass-1740,Zenyatta,0.0
1870786,FlashedDusk-2754,Zenyatta,0.0


## 3. Role Category Implementation

In [4]:
# Create Role category for each Hero;
tanks = ['D.Va','Orisa', 'Reinhardt', 'Roadhog', 'Sigma', 'Winston', 'Wrecking Ball', 'Zarya']
damage = ['Ashe','Bastion','Doomfist','Genji','Hanzo','Junkrat','McCree','Mei','Pharah','Reaper','Soldier: 76','Sombra','Symmetra','Torbjörn','Tracer','Widowmaker']
support = ['Ana','Baptiste','Brigitte','Lúcio','Mercy','Moira','Zenyatta']

# Create list of all heroes to use in predict function later
all_heroes = tanks+damage+support
# create roles df and apply role column adding in the role of the hero. To be used in interactive input function later
roles = heroes.copy()
roles['Role'] = ['Tank' if x in tanks else 'Damage' if x in damage else 'Support' for x in heroes['Hero']] 


In [5]:
# check the state of the dataframes: 
# heroes
# roles

## 4. Implementing the Recommendation system

In [6]:
# importing relevant libraries
from surprise import Reader, Dataset, accuracy
from surprise.model_selection import train_test_split

# from surprise.model_selection import cross_validate
# from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
# from surprise.model_selection import GridSearchCV


#Read in Values as a Surprise Dataset:
reader = Reader(rating_scale=(1,5))

train_data = Dataset.load_from_df(heroes[['Battle_Tag', 'Hero', 'Ratings']], reader)

# Train test split the dataset 
trainset, testset = train_test_split(train_data, test_size=.25)


### 4.1 Training the Model

In [5]:
# # Instantiate KNN Baseline memory based model:
# # ⏰ This cell may take a couple of minutes to run

algobaseitems = KNNBaseline(sim_options={'name':'cosine', 'user_based':False})
algobaseitems.fit(trainset)

# Evaluating the model performance on testset and observing the model accuracy:
predictions = algobaseitems.test(testset)
accuracy.rmse(predictions)


NameError: name 'trainset' is not defined

## 5. Making a Recommendation


### 5.1 Interactive User Rating Input

In [8]:
## try out the interactive user rating function here!
user_rating = hero_rater('Newuser2', roles, 5)

             Hero     Role
1838193  Zenyatta  Support


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 5


         Hero    Role
428071  Genji  Damage


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 1


          Hero  Role
1750391  Zarya  Tank


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 5


         Hero  Role
950490  Orisa  Tank


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 4


             Hero    Role
1421058  Symmetra  Damage


How do you rate this Hero on a scale of 1-5?, press n if you do not want to give a rating :
 5


In [9]:
# check the format of our user_rating
user_rating

[{'Battle_Tag': 'Newuser2', 'Hero': 'Zenyatta', 'Ratings': '5'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Genji', 'Ratings': '1'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Zarya', 'Ratings': '5'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Orisa', 'Ratings': '4'},
 {'Battle_Tag': 'Newuser2', 'Hero': 'Symmetra', 'Ratings': '5'}]

### 5.2 Making Predictions with the new user ratings


In [15]:
# Call our prediction function to predict a new hero for a new user
prediction(True, user_rating, heroes, algobaseitems, all_heroes, reader)

['Mercy', 'Moira', 'Orisa', 'Reinhardt', 'Winston']


[('Widowmaker', 1.1902253735881367),
 ('D.Va', 1.128029866688412),
 ('Ana', 1.0694060701969643),
 ('Hanzo', 1.0239084283666222),
 ('Genji', 0.9557899106566432),
 ('McCree', 0.8964564280769183),
 ('Lúcio', 0.8434863775075112),
 ('Zenyatta', 0.8072269794176803),
 ('Soldier: 76', 0.788818310085804),
 ('Roadhog', 0.6632566388655392),
 ('Brigitte', 0.6079452210204122),
 ('Junkrat', 0.6042837495737843),
 ('Zarya', 0.5246616770865432),
 ('Mei', 0.44627281879928665),
 ('Tracer', 0.4419836940485421),
 ('Pharah', 0.43270907168780937),
 ('Symmetra', 0.42751362987780417),
 ('Baptiste', 0.3842956779515646),
 ('Bastion', 0.34846166754378877),
 ('Sigma', 0.2947251408349301),
 ('Reaper', 0.27000918857572),
 ('Torbjörn', 0.2265075961635747),
 ('Wrecking Ball', 0.20937704959807615),
 ('Sombra', 0.05031639482956751),
 ('Ashe', -0.036750109464269015),
 ('Doomfist', -0.3020762525752598)]